In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
import math
import random
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score,roc_auc_score,auc,roc_curve
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

In [2]:
#data1 = pd.read_csv("../dataset/NASA/CM1.csv")
#data1.head()

In [3]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import datetime
import csv

def show_accuracy(predictLabel,Label):
    Label = np.ravel(Label).tolist()
    predictLabel = predictLabel.tolist()
    count = 0
    for i in range(len(Label)):
        if Label[i] == predictLabel[i]:
            count += 1
    return (round(count/len(Label),5))

class node_generator(object):
    def __init__(self, whiten = False):
        self.Wlist = []
        self.blist = []
        self.function_num = 0
        self.whiten = whiten

    def sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))

    def relu(self, x):
        return np.maximum(x, 0)

    def tanh(self, x):
        return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def linear(self, x):
        return x

    def orth(self, W):
        """
        目前看来，这个函数应该配合下一个generator函数是生成权重的
        """
        for i in range(0, W.shape[1]):
            w = np.mat(W[:,i].copy()).T
            w_sum = 0
            for j in range(i):
                wj = np.mat(W[:,j].copy()).T
                w_sum += (w.T.dot(wj))[0,0]*wj
            w -= w_sum
            w = w/np.sqrt(w.T.dot(w))
            W[:,i] = np.ravel(w)

        return W

    def generator(self, shape, times):
        for i in range(times):
            #random.seed(i)
            W = 2*np.random.random(size=shape)-1
            if self.whiten == True:
                W = self.orth(W)   # 只在增强层使用
            b = 2*np.random.random() -1
            yield (W, b)

    def generator_nodes(self, data, times, batchsize, function_num):
        # 按照bls的理论，mapping layer是输入乘以不同的权重加上不同的偏差之后得到的
        # 若干组，所以，权重是一个列表，每一个元素可作为权重与输入相乘
        self.Wlist = [elem[0] for elem in self.generator((data.shape[1], batchsize), times)]
        self.blist = [elem[1] for elem in self.generator((data.shape[1], batchsize), times)]

        self.function_num = {'linear':self.linear,
                        'sigmoid': self.sigmoid,
                        'tanh':self.tanh,
                        'relu':self.relu }[function_num]  # 激活函数供不同的层选择
        # 下面就是先得到一组mapping nodes，再不断叠加，得到len(Wlist)组mapping nodes
        nodes = self.function_num(data.dot(self.Wlist[0]) + self.blist[0])
        for i in range(1, len(self.Wlist)):
            nodes = np.column_stack((nodes, self.function_num(data.dot(self.Wlist[i])+self.blist[i])))
        return nodes

    def transform(self,testdata):
        testnodes = self.function_num(testdata.dot(self.Wlist[0])+self.blist[0])
        for i in range(1,len(self.Wlist)):
            testnodes = np.column_stack((testnodes, self.function_num(testdata.dot(self.Wlist[i])+self.blist[i])))
        return testnodes

class scaler:
    def __init__(self):
        self._mean = 0
        self._std = 0
    
    def fit_transform(self,traindata):
        self._mean = traindata.mean(axis = 0)
        self._std = traindata.std(axis = 0)
        return (traindata-self._mean)/(self._std+0.001)
    
    def transform(self,testdata):
        return (testdata-self._mean)/(self._std+0.001)

class broadnet:
    def __init__(self, 
                 maptimes = 20, 
                 enhencetimes = 20,
                 map_function = 'linear',
                 enhence_function = 'linear',
                 batchsize = 'auto', 
                 reg = 0.001):
        
        self._maptimes = maptimes
        self._enhencetimes = enhencetimes
        self._batchsize = batchsize
        self._reg = reg
        self._map_function = map_function
        self._enhence_function = enhence_function
        
        self.W = 0
        self.pesuedoinverse = 0
        self.normalscaler = scaler()
        self.onehotencoder = preprocessing.OneHotEncoder(sparse = False)
        self.mapping_generator = node_generator()
        self.enhence_generator = node_generator(whiten = True)

    def fit(self,data,label):
        if self._batchsize == 'auto':
            self._batchsize = data.shape[1]
        data = self.normalscaler.fit_transform(data)
        label = self.onehotencoder.fit_transform(np.mat(label).T)
        
        mappingdata = self.mapping_generator.generator_nodes(data,self._maptimes,self._batchsize,self._map_function)
        enhencedata = self.enhence_generator.generator_nodes(mappingdata,self._enhencetimes,self._batchsize,self._enhence_function)
        
        #print('number of mapping nodes {0}, number of enhence nodes {1}'.format(mappingdata.shape[1],enhencedata.shape[1]))
        #print('mapping nodes maxvalue {0} minvalue {1} '.format(round(np.max(mappingdata),5),round(np.min(mappingdata),5)))
        #print('enhence nodes maxvalue {0} minvalue {1} '.format(round(np.max(enhencedata),5),round(np.min(enhencedata),5)))
        
        inputdata = np.column_stack((mappingdata,enhencedata))
        pesuedoinverse = self.pinv(inputdata,self._reg)
        #print('pesuedoinverse:', pesuedoinverse)
        self.W =  pesuedoinverse.dot(label)

        
        #print('W:', self.W)
        #print('W:', self.W.shape)  
    
    def pinv(self,A,reg):
        return np.mat(reg*np.eye(A.shape[1])+A.T.dot(A)).I.dot(A.T)
    
    def decode(self,Y_onehot):
        Y = []
        for i in range(Y_onehot.shape[0]):
            lis = np.ravel(Y_onehot[i,:]).tolist()
            Y.append(lis.index(max(lis)))
        return np.array(Y)
    
    def accuracy(self,predictlabel,label):
        label = np.ravel(label).tolist()
        predictlabel = predictlabel.tolist()
        count = 0
        for i in range(len(label)):
            if label[i] == predictlabel[i]:
                count += 1
        return (round(count/len(label),5))
        
    def predict(self,testdata):
        testdata = self.normalscaler.transform(testdata)
        test_mappingdata = self.mapping_generator.transform(testdata)
        test_enhencedata = self.enhence_generator.transform(test_mappingdata)
        
        test_inputdata = np.column_stack((test_mappingdata,test_enhencedata)) 
        #print('*predictlabel shape:',self.decode(test_inputdata.dot(self.W)).shape)
        #print('*predictlabel:', self.decode(test_inputdata.dot(self.W)))
        #print('*accuracy:',show_accuracy(self.decode(test_inputdata.dot(self.W)),testlabel))
        return self.decode(test_inputdata.dot(self.W))      

In [4]:
if __name__ == '__main__':
    data = pd.read_csv("../../dataset/NASA/PC4.csv")  
  
    le = preprocessing.LabelEncoder()
    for item in data.columns:
        data[item] = le.fit_transform(data[item])
    #print(data)
    
    label = data['c'].values
    
    data = data.drop('c',axis=1)
    data = data.values
    #print(data.shape,max(label)+1)

    Evaluat_list=[]
    for i in np.arange(0,10):
        #将数据集划分为训练集和测试集
        traindata,testdata,trainlabel,testlabel = train_test_split(data,label,test_size=0.25,random_state = i)
        bls = broadnet(maptimes = 15, 
                   enhencetimes = 25,
                   map_function = 'relu',
                   enhence_function = 'relu',
                   batchsize =100,
                   reg = 0.001)

        #训练
        starttime = datetime.datetime.now()
        bls.fit(traindata,trainlabel)
        endtime = datetime.datetime.now()
        trainingTime=(endtime - starttime).total_seconds()
        #print('the training time of BLS is {0} seconds'.format((endtime - starttime).total_seconds()))

        #预测
        predictlabel = bls.predict(testdata)

        #评价指标计算
        acc=accuracy_score(testlabel,predictlabel,normalize=True)
        fmeasure=f1_score(testlabel,predictlabel, average='weighted', labels=np.unique(testlabel))
        try:
            auc=roc_auc_score(testlabel,predictlabel, average='macro', sample_weight=None)
        except ValueError:
            pass
        MCC=matthews_corrcoef(testlabel,predictlabel)
        Gmeasure=geometric_mean_score(testlabel,predictlabel, average='weighted')
        recall=recall_score(testlabel, predictlabel, average='weighted')

        print('trainingTime：%f,acc：%f,fmeasure：%f,auc：%f,recall：%f,MCC：%f,Gmeasure：%f'%(trainingTime,acc,fmeasure,auc,recall,MCC,Gmeasure))
        Evaluat_tuple=(trainingTime,acc,fmeasure,auc,recall,MCC,Gmeasure)    
        Evaluat_list.append(Evaluat_tuple)

    # 表头
    header = ['trainTime', 'acc', 'fmeasure', 'auc','recall', 'MCC', 'Gmeasure']
    with open('./data_remember/0.75BLS_PC4_Evaluat.csv', 'w', encoding='utf-8', newline='') as file_obj:
        # 创建对象
        writer = csv.writer(file_obj)
        # 写表头
        writer.writerow(header)
        # 3.写入数据(一次性写入多行)
        writer.writerows(Evaluat_list)


        #sio.savemat('./data_remember/WBLS_Smote_CM1_test.mat',{'time':format((endtime - starttime).total_seconds()),'acc':acc,'f1':fmeasure,'auc':auc,'mcc':MCC,'Gm':Gmeasure})

trainingTime：3.677754,acc：0.868493,fmeasure：0.871252,auc：0.765543,recall：0.868493,MCC：0.509242,Gmeasure：0.758589
trainingTime：3.654599,acc：0.841096,fmeasure：0.852056,auc：0.686804,recall：0.841096,MCC：0.324988,Gmeasure：0.669248
trainingTime：3.613913,acc：0.895890,fmeasure：0.893672,auc：0.729416,recall：0.895890,MCC：0.478824,Gmeasure：0.710165
trainingTime：3.601211,acc：0.884932,fmeasure：0.890017,auc：0.773581,recall：0.884932,MCC：0.500683,Gmeasure：0.765525
trainingTime：3.626097,acc：0.893151,fmeasure：0.893584,auc：0.781878,recall：0.893151,MCC：0.559201,Gmeasure：0.773920
trainingTime：3.617245,acc：0.857534,fmeasure：0.865866,auc：0.758872,recall：0.857534,MCC：0.460143,Gmeasure：0.752431
trainingTime：3.610372,acc：0.854795,fmeasure：0.856902,auc：0.665824,recall：0.854795,MCC：0.322155,Gmeasure：0.638445
trainingTime：3.574019,acc：0.852055,fmeasure：0.859172,auc：0.720015,recall：0.852055,MCC：0.399460,Gmeasure：0.707804
trainingTime：3.622659,acc：0.887671,fmeasure：0.893344,auc：0.776950,recall：0.887671,MCC：0.500405,G

In [ ]:
# acc=accuracy_score(predictlabel, testlabel, normalize=True)
# precision=precision_score(predictlabel, testlabel, average='weighted',zero_division=1)
# recall=recall_score(predictlabel, testlabel, average='weighted')
# fmeasure=f1_score(predictlabel, testlabel, average='weighted', labels=np.unique(testlabel))
# auc=roc_auc_score(predictlabel, testlabel, average='weighted', sample_weight=None)
# MCC=matthews_corrcoef(predictlabel, testlabel)
# Gmeasure=geometric_mean_score(predictlabel, testlabel, average='weighted')

In [ ]:
# print('acc：%f,precision：%f,recall：%f,fmeasure：%f,auc：%f,MCC：%f,Gmeasure：%f'%(acc,precision,recall,fmeasure,auc,MCC,Gmeasure))